In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2017)

ImportError: cannot import name 'api_key' from 'config' (C:\Users\Xena\Documents\Bootcamp\NUCHI201905DATA2-master\NUCHI201905DATA2\07-Project-1\DataScience_Project1\Census_Data\config.py)

In [108]:
zipcode_df=pd.read_csv("zipcode.csv")
zipcode_df['Zipcode'] = pd.to_numeric(zipcode_df['Zipcode'])
zipcode_df.head()

,Zipcode
0,60645
1,60626
2,60631
3,60646
4,60659


In [106]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels

census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})



# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

   

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]
census_pd['Zipcode'] = pd.to_numeric(census_pd['Zipcode'])
census_pd.head()

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,601,17599.0,38.9,11757.0,7041.0,11282.0,64.105915
1,602,39209.0,40.9,16190.0,8978.0,20428.0,52.100283
2,603,50135.0,40.4,16645.0,10897.0,25176.0,50.216416
3,606,6304.0,42.8,13387.0,5960.0,4092.0,64.911168
4,610,27590.0,41.4,18741.0,9266.0,12553.0,45.498369


In [109]:
chicago_df = pd.merge(zipcode_df, census_pd, on="Zipcode")
chicago_df.head()

# len(chicago_df)

,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate
0,60645,48269.0,35.5,50589.0,26150.0,9858.0,20.423046
1,60626,49416.0,33.9,39163.0,26849.0,11450.0,23.170633
2,60631,29566.0,42.8,82226.0,43669.0,1505.0,5.090306
3,60646,27865.0,44.3,86534.0,46488.0,1694.0,6.079311
4,60659,40442.0,34.1,50554.0,22921.0,8599.0,21.262549


In [110]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
chicago_df.to_csv("chicago_census_data.csv", encoding="utf-8", index=False)